In [ ]:
import os
import subprocess
import time
import random
import copy
import numpy as np
import renom as rm
import renom_img
from datetime import datetime
from renom_img.api.detection.ssd import SSD
from renom_img.api.utility.augmentation import Augmentation
from renom_img.api.utility.augmentation.process import *
from renom_img.api.utility.load import parse_xml_detection, load_img 
from renom_img.api.utility.distributor.distributor import ImageDistributor
from renom.cuda import set_cuda_active
set_cuda_active(True)

In [ ]:
os.mkdir('model_logs')
date = str(datetime.date(datetime.now()))

In [ ]:
def create_list(filename):
    ann=[]
    img_path=[]
    with open(filename,'r') as f:
        for line in f:
            line = line[:-1]
            line = "/mnt/research/dataset/VOCdevkit/"+line
            img_path.append(line)
            line = line.replace('JPEGImages','Annotations')
            line = line.replace('jpg','xml')
            ann.append(line)
    return img_path,ann

In [ ]:
def reconstruct_box(pred,actual):
    for i in range(len(pred)):
        h = actual[i][0]['size'][1]
        w = actual[i][0]['size'][0]
        if len(pred[i])>0:
            for j in range(len(pred[i])):
                xmin = pred[i][j]['box'][0] * w
                ymin = pred[i][j]['box'][1] * h
                xmax = pred[i][j]['box'][2] * w
                ymax = pred[i][j]['box'][3] * h
                pred[i][j]['box']=[xmin, ymin, xmax, ymax]
    return pred

In [ ]:
def end_function(*args):
    if len(args)>0:
#         calculating mAP
        model = args[1]
#         saving model
        train_list = args[2]
        validation_loss_list = args[3]
        epoch = args[0]
        if len(validation_loss_list)>1:
            tmp = copy.deepcopy(validation_loss_list)
            current_loss = tmp[-1]
            del(tmp[-1])
            tmp.sort()
            if(current_loss<tmp[0]):
                test_dist = ImageDistributor(valid_image)
                results = []
                for i, (x_img_list, _) in enumerate(test_dist.batch(1, shuffle=False)):
                    img_array = np.vstack([load_img(path, model.imsize)[None]
                                           for path in x_img_list])
                    img_array = model.preprocess(img_array)
                    results.extend(model.get_bbox(model(img_array).as_ndarray(),
                                                 score_threshold=0.005, nms_threshold=0.45))

                predicted = reconstruct_box(results,valid_annotation)
                ev = EvaluatorDetection(predicted,valid_annotation)
                fp = open('model_logs/ssd@'+date+'.txt','a+')
                fp.write('Epoch: {:03d} Train Loss: {:3.2f}  Valid Loss: {:3.2f} mAP: {:3.2f} \n'.format(epoch,float(train_list[-1]),float(validation_loss_list[-1]),float(ev.mAP())))
                fp.close()

In [ ]:
train_image, train_annot = create_list("/mnt/research/dataset/VOCdevkit/voc_train.txt")
valid_image, valid_annot = create_list("/mnt/research/dataset/VOCdevkit/2007_test.txt")

train_annotation, cmap = parse_xml_detection(train_annot)
valid_annotation, _ = parse_xml_detection(valid_annot)

In [ ]:
aug=Augmentation([
        RandomBrightness(),
        ContrastNorm(),
        RandomHue(),
        RandomSaturation(),
        RandomLighting(),
        RandomExpand(),
        RandomCrop(),
        Flip(),
    ])

In [ ]:
model = SSD(cmap, load_pretrained_weight=True, train_whole_network=True)

In [ ]:
# Hyperparameters
total_epoch = 160
batch = 8
imsize = model.imsize
multiscale = None
optimizer = model._opt.__class__
augmentation = [str(name.__class__).split('.')[-1] for name in aug._process_list]
evaluation_matrix = "mAP"
dataset = "PASCAL_VOC_2012"
standard = 77.5
load_pretrained=True
train_whole=True
renom_v = rm.__version__
renom_img_v = renom_img.__version__
commit_id = str(subprocess.check_output(['git','rev-parse','HEAD']))

In [ ]:
# write hyperparameters to file
fp = open('model_logs/ssd@'+date+'.txt','a+')
fp.write('Commit Hash: '+commit_id[2:-3]+'\nReNom version: '+renom_v+'\nReNomIMG version: '+renom_img_v)
fp.write('\nExpected score: {:3.2f}\n'.format(float(standard)))
fp.write('\n===================================================Hyperparameters=======================================================\n')
fp.write('\nTotal epoch: {:03d}\nBatch size: {:03d}\nImage size: ({:03d},{:03d})'.format(total_epoch,batch,imsize[0],imsize[1]))
fp.write('\nMultiscale: '+str(multiscale)+'\nOptimizer: '+str(optimizer).split('.')[-1]+'\nAugmentation: '+str(augmentation))
fp.write('\nEvaluation matrix: '+str(evaluation_matrix)+'\nDataset: '+str(dataset))
fp.write('\nLoad Pretrained weight: '+str(load_pretrained)+'\nTrain whole network: '+str(train_whole))
fp.write('\n==========================================================================================================================\n\n')
fp.close()

In [ ]:
model.fit(train_image,train_annotation,valid_image,valid_annotation, augmentation=aug,
                epoch=total_epoch,batch_size=batch,callback_end_epoch=end_function)
fp = open('model_logs/ssd@'+date+'.txt','a')
fp.write('\nSuccess')
fp.close()